In [322]:
using IJulia
using Ket
using JuMP
import LinearAlgebra
import SCS
import Hypatia

In [323]:
k, d = 3, 4
ρ = Ket.random_state(d, k)

4×4 LinearAlgebra.Hermitian{ComplexF64, Matrix{ComplexF64}}:
   0.196321+0.0im        -0.0181549+0.152488im   …  0.225009+0.0756234im
 -0.0181549-0.152488im     0.176871+0.0im           0.131285-0.137434im
 -0.0237721+0.0800261im  -0.0110952-0.0342395im     0.055514+0.10994im
   0.225009-0.0756234im    0.131285+0.137434im      0.487017+0.0im

In [324]:
X = ρ

4×4 LinearAlgebra.Hermitian{ComplexF64, Matrix{ComplexF64}}:
   0.196321+0.0im        -0.0181549+0.152488im   …  0.225009+0.0756234im
 -0.0181549-0.152488im     0.176871+0.0im           0.131285-0.137434im
 -0.0237721+0.0800261im  -0.0110952-0.0342395im     0.055514+0.10994im
   0.225009-0.0756234im    0.131285+0.137434im      0.487017+0.0im

In [325]:
X = Matrix(X)
if !LinearAlgebra.isposdef(X)
    error("X is not positive semidefinite, so the idea of it being separable does not make sense.")
end

In [326]:
lX = maximum(size(X))
rX = LinearAlgebra.rank(X)
X = X / LinearAlgebra.tr(X)
sep = -1

-1

In [327]:
dim = round(sqrt(lX))
str = 2
verbose = 1
tol = eps()^(3 / 8)

1.3486991523486091e-6

In [328]:
if length(dim) == 1
    dim = [dim, lX / dim]
    if abs(dim[2] - round(dim[2])) >= 2 * lX * eps()
        error("IsSeparable:InvalidDim','If DIM is a scalar, it must evenly divide length(X); please provide the DIM array containing the dimensions of the subsystems.")
    end
    dim[2] = round(dim[2])
end

2.0

In [329]:
nD = minimum(dim)
xD = maximum(dim)
pD = prod(dim)

4.0

In [330]:
if nD == 1
    sep = 1
    if Bool(verbose)
        println("Every positive semidefinite matrix is separable when one of the local dimensions is 1.")
    end

    return
end

In [331]:
XA = Ket.partial_trace(X, [2], Int.(dim))
XB = Ket.partial_trace(X, [1], Int.(dim))

2×2 Matrix{ComplexF64}:
  0.336111+0.0im       0.0373591+0.262428im
 0.0373591-0.262428im   0.663889+0.0im

In [332]:
refs = [
    "A. Peres. Separability criterion for density matrices. Phys. Rev. Lett., 77:1413–1415, 1996.",
    "M. Horodecki, P. Horodecki, and R. Horodecki. Separability of mixed states: Necessary and sufficient conditions. Phys. Lett. A, 223:1–8, 1996.",
    "P. Horodecki, M. Lewenstein, G. Vidal, and I. Cirac. Operational criterion and constructive checks for the separability of low-rank density matrices. Phys. Rev. A, 62:032310, 2000.",
    "K. Chen and L.-A. Wu. A matrix realignment method for recognizing entanglement. Quantum Inf. Comput., 3:193–202, 2003.",
    "F. Verstraete, J. Dehaene, and B. De Moor. Normal forms and entanglement measures for multipartite quantum states. Phys. Rev. A, 68:012103, 2003.",
    "K.-C. Ha and S.-H. Kye. Entanglement witnesses arising from exposed positive linear maps. Open Systems & Information Dynamics, 18:323–337, 2011.",
    "O. Gittsovich, O. Guehne, P. Hyllus, and J. Eisert. Unifying several separability conditions using the covariance matrix criterion. Phys. Rev. A, 78:052319, 2008.",
    "L. Gurvits and H. Barnum. Largest separable balls around the maximally mixed bipartite quantum state. Phys. Rev. A, 66:062311, 2002.",
    "H.-P. Breuer. Optimal entanglement criterion for mixed quantum states. Phys. Rev. Lett., 97:080501, 2006.",
    "W. Hall. Constructions of indecomposable positive maps based on a new criterion for indecomposability. E-print: arXiv:quant-ph/0607035, 2006.",
    "A. C. Doherty, P. A. Parrilo, and F. M. Spedalieri. A complete family of separability criteria. Phys. Rev. A, 69:022308, 2004.",
    "M. Navascues, M. Owari, and M. B. Plenio. A complete criterion for separability detection. Phys. Rev. Lett., 103:160404, 2009.",
    "N. Johnston. Separability from spectrum for qubit-qudit states. Phys. Rev. A, 88:062330, 2013.",
    "C.-J. Zhang, Y.-S. Zhang, S. Zhang, and G.-C. Guo. Entanglement detection beyond the cross-norm or realignment criterion. Phys. Rev. A, 77:060301(R), 2008.",
    "R. Hildebrand. Semidefinite descriptions of low-dimensional separable matrix cones. Linear Algebra Appl., 429:901–932, 2008.",
    "R. Hildebrand. Comparison of the PPT cone and the separable cone for 2-by-n systems. http://www-ljk.imag.fr/membres/Roland.Hildebrand/coreMPseminar2005_slides.pdf",
    "D. Cariello. Separability for weak irreducible matrices. E-print: arXiv:1311.7275 [quant-ph], 2013.",
    "L. Chen and D. Z. Djokovic. Separability problem for multipartite states of rank at most four. J. Phys. A: Math. Theor., 46:275304, 2013.",
    "G. Vidal and R. Tarrach. Robustness of entanglement. Phys. Rev. A, 59:141–155, 1999."
]


19-element Vector{String}:
 "A. Peres. Separability criterio" ⋯ 30 bytes ⋯ "Rev. Lett., 77:1413–1415, 1996."
 "M. Horodecki, P. Horodecki, and" ⋯ 80 bytes ⋯ ". Phys. Lett. A, 223:1–8, 1996."
 "P. Horodecki, M. Lewenstein, G." ⋯ 119 bytes ⋯ "Phys. Rev. A, 62:032310, 2000."
 "K. Chen and L.-A. Wu. A matrix " ⋯ 56 bytes ⋯ " Inf. Comput., 3:193–202, 2003."
 "F. Verstraete, J. Dehaene, and " ⋯ 83 bytes ⋯ " Phys. Rev. A, 68:012103, 2003."
 "K.-C. Ha and S.-H. Kye. Entangl" ⋯ 82 bytes ⋯ "ion Dynamics, 18:323–337, 2011."
 "O. Gittsovich, O. Guehne, P. Hy" ⋯ 101 bytes ⋯ "Phys. Rev. A, 78:052319, 2008."
 "L. Gurvits and H. Barnum. Large" ⋯ 70 bytes ⋯ " Phys. Rev. A, 66:062311, 2002."
 "H.-P. Breuer. Optimal entanglem" ⋯ 43 bytes ⋯ "s. Rev. Lett., 97:080501, 2006."
 "W. Hall. Constructions of indec" ⋯ 79 bytes ⋯ ": arXiv:quant-ph/0607035, 2006."
 "A. C. Doherty, P. A. Parrilo, a" ⋯ 64 bytes ⋯ " Phys. Rev. A, 69:022308, 2004."
 "M. Navascues, M. Owari, and M. " ⋯ 64 bytes ⋯ ". Rev. Lett., 103:1604

In [333]:
function is_ppt(ρ::AbstractMatrix, dims::Vector{Int}, sys::Int=2; atol=1e-10)
    ρ_pt = partial_transpose(ρ, [sys], dims)
    λs = LinearAlgebra.eigen(ρ_pt).values
    return all(λs .>= -atol)
end

ppt = is_ppt(X, Int.(dim))


false

In [334]:
if !ppt
    sep = 0
    if Bool(verbose)
        println("Determined to be entangled via the PPT criterion. Reference:\n", refs[1], "\n")
    end
    return

elseif pD <= 6 || minimum(dim) <= 1
    sep = 1
    if Bool(verbose)
        println("Determined to be separable via sufficiency of the PPT criterion in small dimensions. Reference:\n", refs[2], "\n")
    end
    return

elseif rX <= 3 || rX <= LinearAlgebra.rank(XB) || rX <= LinearAlgebra.rank(XA)
    sep = 1
    if Bool(verbose)
        ("Determined to be separable via sufficiency of the PPT criterion for low-rank operators. Reference:\n", refs[3], "\n")
    end
    return
end

Determined to be entangled via the PPT criterion. Reference:
A. Peres. Separability criterion for density matrices. Phys. Rev. Lett., 77:1413–1415, 1996.



In [337]:
"""
    realignment(X::AbstractArray, dim=nothing)

Performs the realignment transformation on a bipartite quantum operator `X`.

# Arguments
- `X`: A square matrix representing the bipartite system.
- `dim`: Optional. A two-element array specifying the dimensions of the subsystems. 
         If not provided, an attempt is made to infer it from `X`.

# Returns
- A matrix of size `dA^2 × dB^2` where `dA` and `dB` are subsystem dimensions.

# Notes
- If `dim` is a scalar, it's interpreted as `dA` and `dB` is inferred.
- Throws an error if dimensions are incompatible.
"""
function realignment(X::AbstractArray, dim=nothing)
    dX = size(X)
    inferred_dim = round.(sqrt.(dX))

    if dim === nothing
        dim = collect(inferred_dim)
    end

    if length(dim) == 1
        dim = [dim, dX[1] / dim]
        if abs(dim[2] - round(dim[2])) >= 2 * dX[1] * eps()
            error("Realignment:InvalidDim — ",
                "If `dim` is a scalar, `X` must be square and `dim` must evenly divide length(X); ",
                "please provide a two-element `dim` array specifying the subsystem dimensions.")
        end
        dim[2] = round(dim[2])
    end

    if minimum(size(dim)) == 1
        dim = vec(dim)'
        dim = vcat(dim, dim)
    end

    dA, dB = Int.(vec(dim))

    X_tensor = reshape(X, dA, dB, dA, dB)
    X_swapped = permutedims(X_tensor, (2, 1, 4, 3))
    X_swapped_matrix = reshape(X_swapped, dA * dB, dA * dB)

    X_pt = Ket.partial_transpose(X_swapped_matrix, [1], [dB, dA])
    X_pt_tensor = reshape(X_pt, dB, dA, dB, dA)
    X_unswapped = permutedims(X_pt_tensor, (2, 1, 4, 3))

    return reshape(X_unswapped, dA^2, dB^2)
end


realignment

In [338]:
if Bool(verbose) && Ket.trace_norm(realignment(X, dim)) > 1
    sep = 0
    println("Determined to be entangled via the realignment criterion. Reference:\n$(refs[4])\n")
    return
end

Determined to be entangled via the realignment criterion. Reference:
K. Chen and L.-A. Wu. A matrix realignment method for recognizing entanglement. Quantum Inf. Comput., 3:193–202, 2003.



In [339]:
if Ket.trace_norm(realignment(X - kron(XA, XB))) > sqrt(real((1 - LinearAlgebra.tr(XA^2)) * (1 - LinearAlgebra.tr(XB^2))))
    sep = 0
    println("Determined to be entangled by using Theorem 1 of reference:\n", refs[14], "\n")
    return
end

Determined to be entangled by using Theorem 1 of reference:
C.-J. Zhang, Y.-S. Zhang, S. Zhang, and G.-C. Guo. Entanglement detection beyond the cross-norm or realignment criterion. Phys. Rev. A, 77:060301(R), 2008.



In [340]:
lam = sort(real.(LinearAlgebra.eigen(X).values), rev=true)

4-element Vector{Float64}:
 0.742717740775585
 0.23062359488903747
 0.02665866433537675
 1.0656071716582526e-16

In [341]:
function swap_subsystems(X::Matrix{<:Number}, sys::Tuple{Int,Int}, dim::AbstractVector{<:Int})
    # Ensure dim is a 1D vector of length 2
    d1, d2 = Int.(vec(dim))
    total_dim = d1 * d2
    @assert size(X, 1) == total_dim && size(X, 2) == total_dim "Matrix size doesn't match subsystem dimensions"

    # Reshape X into 4D tensor: (a1, a2, b1, b2)
    X_tensor = reshape(X, d1, d2, d1, d2)

    # Swap subsystems 1 and 2: (a1, a2) → (a2, a1) and (b1, b2) → (b2, b1)
    # permuted order: (2, 1, 4, 3)
    X_swapped = permutedims(X_tensor, (2, 1, 4, 3))

    # Reshape back to matrix
    return reshape(X_swapped, total_dim, total_dim)
end

swap_subsystems (generic function with 1 method)

In [342]:
if nD == 2
    if ((lam[1] - lam[Int(2 * xD - 1)])^2) <= (4 * lam[Int(2 * xD - 2)] * lam[Int(2 * xD)] + tol^2)
        sep = 1
        if Bool(verbose)
            println("Determined to be separable by inspecting its eigenvalues. Reference:\n", refs[13], "\n")
        end
        return
    end
    if dim[1] > 2
        Xt = swap_subsystems(X, (1, 2), dim)
    else
        Xt = X
    end

    A = Xt[1:Int(xD), 1:Int(xD)]
    B = Xt[1:Int(xD), Int(xD)+1:2*Int(xD)]
    C = Xt[Int(xD)+1:2*Int(xD), Int(xD)+1:2*Int(xD)]

    if Bool(verbose) && (LinearAlgebra.norm(B - B') < tol ^ 2)
        sep = 1
        println("Determined to be separable by being a block Hankel matrix:\n", refs[15], "\n")
        return
    end

    if (LinearAlgebra.rank(B - B') <= 1) && ppt
        sep = 1
        if Bool(verbose)
            println("Determined to be separable by being a perturbed block Hankel matrix:\n", refs[16], "\n")
        end
        return
    end

    X_2n_ppt_check = vcat(
        hcat((5/6) * A - C / 6, B),
        hcat(B', (5/6) * C - A / 6)
    )

    if LinearAlgebra.isposdef(X_2n_ppt_check) && is_ppt(X_2n_ppt_check, [2, Int(xD)])
        sep = 1
        if Bool(verbose)
            println("Determined to be separable via the homothetic images approach of:\n", refs[16], "\n")
        end
        return
    end
end